In [ ]:
!pip3 install -q http://download.pytorch.org/whl/cu90/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [1]:
# IMPORT LIBRARIES FOR PROJECT
%matplotlib inline
import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import numpy as np

In [33]:
# CUSTOM FUNCTIONS
import random

def datasetBalancing(originalDict):
    """
    This function has the purpose of balancing the dataset before using it in CNN.
    
    Starting from input data, we separate two distinct classes: one of the original classes
    extracted from the dataset that will act as a leaf (we call it 'class0'), and, excluding 
    the last step (consisting of only the two remaining classes) one "super" class containing 
    all the elements remaining (we call it 'class1')
    
    To keep the dataset balanced, we need the size of class0 to be equal to the size of class1. 
    In order to do that, we compute (based both on the size of class0 and the number of the 
    different classes in class1) the number of elements that have to be extracted from each
    subclass of class1 and we take them after reshuffling the elements to avoid taking
    always the same.
    
    :param (dictionary) originalDict: input dataset
    
    :return (dictionary) reducedDict: the input dataset with one class dropped
    :return (array) values: the values extracted from dataset
    :return (array) labels: the two key, one of one class and the key 'others' for the super class
    """
    
    classToDrop = list(originalDict)[len(originalDict.keys())-1]
    
    class0, class1 = dropClass(originalDict, classToDrop)

    print('Class ' + classToDrop + ' dropped')

    if len(class1.keys()) == 1:
        #TODO: SET A TRESHOLD TO DETERMINE IF ALSO WITH LAST 2 CLASSES WE DON'T KNOW WHICH ONE IS FOR SURE
        dictForCNN = {list(class1.keys())[0]: class1[list(class1.keys())[0]], classToDrop: class0}
        values, labels = returnValues(dictForCNN)
        return class1, values, labels

    elementsFromEachClass = int(round(len(class0) / len(class1.keys())))
    
    print('We will take ' + str(elementsFromEachClass) + ' elements from each class ')
    
    class1_balanced = []
    for k in class1.keys():
        myClassElements = class1[k]
        random.shuffle(myClassElements)
        class1_balanced += myClassElements[:elementsFromEachClass]
    
    dictForCNN = {}
    
    dictForCNN.update({'others': []})
    dictForCNN.update({classToDrop: []})
        
    for i in class1_balanced:
        dictForCNN['others'].append(i)
    for i in class0:
        dictForCNN[classToDrop].append(i)
        
    values, labels = returnValues(dictForCNN)
        
    return class1, values, labels

def dropClass(dictInput, className):
    """
    Function with the aim of dropping one class from a dictionary
    
    :param (dictionary) dictInput: the dictionary
    :param (string) className: the name of the class that has to be dropped from dictInput
    
    :return (array) class0: the elements contained in the dropped class
    :return (dictionary) class1: the remainig dictionary
    """
    
    return dictInput.pop(className), dictInput

def toDict(trainset):
    """
    Function with the goal of taking labels and data putting them in a dictionary.
    We take the data with its label and insert it into a dictionary: in the end, 
    we will have a dictionary with the different classes as keys and all the 
    data with that label as values.
    
    :param (dataset) trainset: the trainset
    
    :return (dictionary) returnDict: the resultant dictionary
    """
    
    returnDict = {}
    
    for data, label in zip(trainset.data, trainset.targets):
        label = str(label)
        if label in returnDict.keys():
            returnDict[label].append(data)
        else:
            returnDict.update({label: []})
            returnDict[label].append(data)
            
    return returnDict

"""
PROBABILMENTE NON SERVE A UN CAZZO

def manageTestset(testset, className):
    returnDict = {'others': [], className: []}
    
    for data, label in zip(testset.data, testset.targets):
        label = str(label)
        if label != className:
            returnDict['others'].append(data)
        else:
            returnDict[className].append(data)
"""
            
def returnValues(dictInput):
    """
    Simply function that takes in input a dictionary and returns two array of values and labels before 
    passing them into the DataLoader
    
    :param (dictionary) dictInput: the input dictionary
    
    :return (array) values: all the values
    :return (array) labels: all the labels
    """
    
    values = np.array(list(dictInput.values())[0] + list(dictInput.values())[1])
    labels = [1]*len(list(dictInput.values())[0]) + [0]*len(list(dictInput.values())[1])
    
    return values, labels

In [ ]:
"""
DATASET CLEANING

Since the patient_id is not useful for classification purpose,
we move the dataset into a folder containing X sub-folders
where X is defined as the number of possible classes.
"""
import os

def refactor_dataset():    
    for count, folder_name in enumerate(os.listdir("./DatasetINPUT/")):
        for file_name in os.listdir("./DatasetINPUT/" + folder_name):
            os.rename(file_name, "./DatasetOUTPUT/" + file_name.split("_")[1] + "/" + file_name)
    os.remove("DatasetINPUT")
    os.rename("DatasestOUTPUT", "Dataset")
            
def make_folders_tree(folder_names):
    for folder_name in folder_names:
        mkdir_if_not_exist(folder_name)
               
@staticmethod
def mkdir_if_not_exist(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
        
make_folders_tree(["AC", "H", "Serr", "T", "V"])
refactor_dataset()

In [ ]:
# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def plot_kernel(model):
    model_weights = model.state_dict()
    fig = plt.figure()
    plt.figure(figsize=(10,10))
    for idx, filt  in enumerate(model_weights['conv1.weight']):
    #print(filt[0, :, :])
        if idx >= 32: continue
        plt.subplot(4,8, idx + 1)
        plt.imshow(filt[0, :, :], cmap="gray")
        plt.axis('off')
    
    plt.show()

def plot_kernel_output(model,images):
    fig1 = plt.figure()
    plt.figure(figsize=(1,1))
    
    img_normalized = (images[0] - images[0].min()) / (images[0].max() - images[0].min())
    plt.imshow(img_normalized.numpy().transpose(1,2,0))
    plt.show()
    output = model.conv1(images)
    layer_1 = output[0, :, :, :]
    layer_1 = layer_1.data

    fig = plt.figure()
    plt.figure(figsize=(10,10))
    for idx, filt  in enumerate(layer_1):
        if idx >= 32: continue
        plt.subplot(4,8, idx + 1)
        plt.imshow(filt, cmap="gray")
        plt.axis('off')
    plt.show()

def test_accuracy(net, dataloader):
  ########TESTING PHASE###########
  
    #check accuracy on whole test set
    correct = 0
    total = 0
    net.eval() #important for deactivating dropout and correctly use batchnorm accumulated statistics
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy of the network on the test set: %d %%' % (accuracy))
    return accuracy

    
n_classes = 2       
      
#function to define the convolutional network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #conv2d first parameter is the number of kernels at input (you get it from the output value of the previous layer)
        #conv2d second parameter is the number of kernels you wanna have in your convolution, so it will be the n. of kernels at output.
        #conv2d third, fourth and fifth parameters are, as you can read, kernel_size, stride and zero padding :)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv_final = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(64 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, n_classes) #last FC for classification 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.pool(self.conv_final(x)))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        #hint: dropout goes here!
        x = self.fc2(x)
        return x

      ####RUNNING CODE FROM HERE:
      
#transform are heavily used to do simple and complex transformation and data augmentation
transform_train = transforms.Compose(
    [
     #transforms.RandomHorizontalFlip(),
     transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_test = transforms.Compose(
    [
     transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_train)

#mydict = toDict(trainset)
trainset_old, trainset.data, trainset.targets = datasetBalancing(toDict(trainset))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=4,drop_last=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=4,drop_last=True)


dataiter = iter(trainloader)

#show images just to understand what is inside the dataset
#images, labels = dataiter.next()
#imshow(torchvision.utils.make_grid(images))

net = CNN()

###OPTIONAL:
#print("####plotting kernels of conv1 layer:####")
#plot_kernel(net)
####


net = net.cuda()

criterion = nn.CrossEntropyLoss().cuda() #it already does softmax computation for use!
optimizer = optim.Adam(net.parameters(), lr=0.0001) #better convergency w.r.t simple SGD :)

###OPTIONAL:
#print("####plotting output of conv1 layer:#####")
#plot_kernel_output(net,images)  
###

########TRAINING PHASE###########
n_loss_print = len(trainloader)  #print every epoch, use smaller numbers if you wanna print loss more often!

n_epochs = 20
for epoch in range(n_epochs):  # loop over the dataset multiple times
    net.train() #important for activating dropout and correctly train batchnorm
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs and cast them into cuda wrapper
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % n_loss_print == (n_loss_print -1):    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / n_loss_print))
            running_loss = 0.0
    test_accuracy(net,testloader)
print('Finished Training')
